# Ensemble Learning

## Initial Imports

In [332]:
import warnings
warnings.filterwarnings('ignore')

In [333]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [334]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [335]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [336]:
# Create our features
df = pd.get_dummies(df,columns=["home_ownership", "verification_status"])
df.head()
X = pd.DataFrame(df.apply(lambda x:pd.to_numeric(x, errors='coerce'))).dropna(how='all', axis=1)

# Create our target
y = pd.DataFrame(df['loan_status'])


In [337]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [338]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
3903,1600.0,0.0881,50.74,35964.0,24.12,0.0,0.0,20.0,0.0,1156.0,...,14292.0,58300.0,15000.0,0,0,1,0,1,0,0
28390,9000.0,0.0756,280.21,41000.0,16.89,1.0,0.0,16.0,0.0,11157.0,...,52587.0,24500.0,40492.0,0,0,1,0,1,0,0
15470,10000.0,0.1033,214.10,112000.0,17.75,0.0,0.0,14.0,0.0,16904.0,...,78944.0,65400.0,80642.0,0,1,0,0,0,0,1
4279,36000.0,0.1033,1167.21,120000.0,19.95,0.0,1.0,13.0,0.0,63248.0,...,73826.0,85200.0,30000.0,0,0,0,1,1,0,0
57514,18000.0,0.0881,570.81,51000.0,19.11,1.0,0.0,16.0,0.0,17586.0,...,126195.0,36500.0,96715.0,0,1,0,0,0,0,1


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [339]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [340]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)

In [341]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_train_scaled_df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,-1.468336,-0.821535,-1.493177,-0.425868,0.125899,-0.305221,-0.654739,1.232955,-0.374566,-0.751677,...,-0.819999,1.061596,-0.799186,-0.097194,-1.053961,2.893159,-0.746210,1.043048,-0.770967,-0.417214
1,-0.749305,-1.080866,-0.697756,-0.385027,-0.256606,1.091575,-0.654739,0.568060,-0.374566,-0.295578,...,-0.152045,-0.196620,-0.298824,-0.097194,-1.053961,2.893159,-0.746210,1.043048,-0.770967,-0.417214
2,-0.652139,-0.506189,-0.926915,0.190774,-0.211107,-0.305221,-0.654739,0.235612,-0.374566,-0.033484,...,0.307682,1.325896,0.489249,-0.097194,0.948802,-0.345643,-0.746210,-0.958729,-0.770967,2.396853
3,1.874185,-0.506189,2.376889,0.255653,-0.094716,-0.305221,0.664658,0.069389,-0.374566,2.080051,...,0.218412,2.062958,-0.504763,-0.097194,-1.053961,-0.345643,1.340106,1.043048,-0.770967,-0.417214
4,0.125191,-0.821535,0.309563,-0.303928,-0.139156,1.091575,-0.654739,0.568060,-0.374566,-0.002381,...,1.131849,0.250084,0.804733,-0.097194,0.948802,-0.345643,-0.746210,-0.958729,-0.770967,2.396853


## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [342]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier
model_brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model_brfc.fit(X_train, y_train)


BalancedRandomForestClassifier(random_state=1)

In [343]:
# Calculated the balanced accuracy score
y_brfc_pred = model_brfc.predict(X_test)
balanced_accuracy_score(y_test, y_brfc_pred)

0.7105395993294372

In [344]:
# Display the confusion matrix
confusion_matrix(y_test, y_brfc_pred)

array([[   61,    40],
       [ 3128, 13976]], dtype=int64)

In [345]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_brfc_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.60      0.82      0.04      0.70      0.48       101
   low_risk       1.00      0.82      0.60      0.90      0.70      0.50     17104

avg / total       0.99      0.82      0.61      0.89      0.70      0.50     17205



In [346]:
# List the features sorted in descending order by feature importance
importances = model_brfc.feature_importances_

sorted(zip(model_brfc.feature_importances_, X.columns), reverse=True)

[(0.0741125277111509, 'total_rec_int'),
 (0.07117217222519692, 'total_rec_prncp'),
 (0.060159498197457494, 'total_pymnt'),
 (0.05931577494455895, 'total_pymnt_inv'),
 (0.04287921202458123, 'last_pymnt_amnt'),
 (0.03465213842967681, 'int_rate'),
 (0.021582459880459082, 'installment'),
 (0.020598040928603872, 'mths_since_recent_inq'),
 (0.02008111285882203, 'dti'),
 (0.01845677517213032, 'out_prncp_inv'),
 (0.01845080889042718, 'max_bal_bc'),
 (0.01665065679356889, 'avg_cur_bal'),
 (0.016550138318902948, 'total_bal_il'),
 (0.015755022456467843, 'total_bc_limit'),
 (0.015585166868612648, 'revol_bal'),
 (0.015558723505551503, 'mths_since_rcnt_il'),
 (0.015534890722867649, 'annual_inc'),
 (0.0153965440816064, 'bc_open_to_buy'),
 (0.015184910709827687, 'total_bal_ex_mort'),
 (0.014938007280220716, 'mo_sin_old_il_acct'),
 (0.01447682280411452, 'out_prncp'),
 (0.014471398748100762, 'bc_util'),
 (0.014402837999485813, 'all_util'),
 (0.014032140038623757, 'mo_sin_old_rev_tl_op'),
 (0.01391174628

### Easy Ensemble Classifier

In [347]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
model_eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
model_eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [348]:
# Calculated the balanced accuracy score
y_eec_pred = model_eec.predict(X_test)
balanced_accuracy_score(y_test, y_eec_pred)

0.7297198732969649

In [349]:
# Display the confusion matrix
confusion_matrix(y_test, y_eec_pred)

array([[   70,    31],
       [ 3996, 13108]], dtype=int64)

In [331]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_eec_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.77      0.03      0.73      0.53       101
   low_risk       1.00      0.77      0.69      0.87      0.73      0.54     17104

avg / total       0.99      0.77      0.69      0.86      0.73      0.53     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    EASY ENSEMBLE CLASSIFIER.

2. Which model had the best recall score?

    BALANCED RANDOM FOREST CLASSIFIER.

3. Which model had the best geometric mean score?

    EASY ENSEMBLE CLASSIFIER.

4. What are the top three features?

    1. total_rec_int
    2. total_rec_prncp
    3. total_pymnt.